In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.widgets import MapView
from arcgis.geometry import Geometry, Point, distance
from arcgis.features import FeatureLayer
from arcgis.geocoding import geocode

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Initialize GIS
gis = GIS("home")

# Create a WebMap
webmap = WebMap()

# Set the basemap
webmap.basemap = "streets-navigation-vector"


search_results = gis.content.search("title:Encuentros de ovni" item_type="Feature Service")
feature_layer_collection = search_results[0]
feature_layer = feature_layer_collection.layers[0]

# Define the URL of the feature layer
#ufo_layer_url = "https://services3.arcgis.com/Fq07Av2pa1e9WKEK/arcgis/rest/services/Encuentros_de_ovni/FeatureServer/0"

# Check if the layer is loaded correctly
#if ufo_layer.properties:
   # print("Layer loaded successfully")

# Create a FeatureLayer object
#ufo_layer = FeatureLayer(ufo_layer_url)


#add layer to the map
#webmap.add_layer(ufo_layer)
webmap.add_layer(feature_layer)

# Define the popup template
popup_template = {
    "title": "{CITY}, {STATE}",
    "content": """
        <div style="font-family: Arial, sans-serif; padding: 10px;">
            <div style="font-size: 18px; font-weight: bold; color: #3366cc; margin-bottom: 10px;">Summary: {SUMMARY}</div>
            <div style="margin-bottom: 5px;"><span style="font-weight: bold; color: #666;">Date:</span> {
                Text(Date({DATE_TIME}, 'DD MMM YYYY'))
            }</div>
            <div style="margin-bottom: 5px;"><span style="font-weight: bold; color: #666;">Time:</span> {
                Text(Date({DATE_TIME}, 'HH:mm'))
            }</div>
            <div style="margin-bottom: 5px;"><span style="font-weight: bold; color: #666;">Duration:</span> {DURATION}</div>
            <div style="margin-bottom: 5px;"><span style="font-weight: bold; color: #666;">Description:</span> {DESCRIPTIO}</div>
        </div>
    """
}

# Set the popup template for the feature layer
#ufo_layer.popupInfo = popup_template
feature_layer.popupInfo = popup_template

# Create MapView widget
map_view = MapView(gis=gis, webmap=webmap, zoom=4, center=[-100.33, 43.69])
#map_view.layout.height = '600px'


# Function to search for a city and find the nearest UFO sighting
def search_city(b):
    city_name = city_input.value
    if city_name:
        geocode_result = geocode(city_name, max_locations=1)
        if geocode_result:
            city_location = geocode_result[0]['location']
            city_point = Point(city_location['x'], city_location['y'], spatial_reference={"wkid": 102100})
            
            # Query the UFO layer for features
            query = ufo_layer.query(return_geometry=True, out_fields="*")
            features = query.features

            if features:
                nearest_feature = find_nearest_feature(features, city_point)
                if nearest_feature:
                    nearest_point = nearest_feature.geometry
                    distance = distance(city_point, nearest_point, "kilometers")

                    map_view.zoom = 10
                    map_view.center = [city_point.x, city_point.y]
                    map_view.popup.show({
                        'title': 'Nearest UFO Sighting',
                        'location': nearest_point,
                        'content': f"""
                            Location: {nearest_feature.attributes['CITY']}<br>
                            State: {nearest_feature.attributes['STATE']}<br>
                            Summary: {nearest_feature.attributes['DESCRIPTIO']}<br>
                            Date: {nearest_feature.attributes['DATE_TIME']}<br>
                            Distance: {distance:.2f} km ({(distance * 0.621371):.2f} miles)
                        """
                    })
                else:
                    display(HTML("<b>No UFO sightings found within the specified distance.</b>"))
            else:
                display(HTML("<b>No UFO sightings found within the specified distance.</b>"))
        else:
            display(HTML("<b>City not found.</b>"))
    else:
        display(HTML("<b>Please enter a city name.</b>"))

# Helper function to find the nearest feature
def find_nearest_feature(features, city_point):
    nearest_feature = None
    min_distance = float('inf')

    for feature in features:
        feature_geometry = feature.geometry
        feature_point = feature_geometry.extent.center if feature_geometry.type != 'point' else feature_geometry
        distance = distance(city_point, feature_point, "kilometers")

        if distance < min_distance:
            min_distance = distance
            nearest_feature = feature

    return nearest_feature

# Create search box and button
city_input = widgets.Text(placeholder="Enter city name")
search_button = widgets.Button(description="Search")
search_button.on_click(search_city)

# Display the widgets and the map
display(widgets.HBox([city_input, search_button]))
display(map_view)
